In [9]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-126178")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-126178
Azure region: southcentralus
Subscription id: 82648f26-b738-43a4-9ebb-f954c9f1ff3a
Resource group: aml-quickstarts-126178


In [15]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4

def cluster_validation():
    cluster = 'my-cluster'
    try:
        compute_cluster = ComputeTarget(workspace = ws, name = cluster)
        print('Found existing compute cluster!')
    except ComputeTargetException:
        config = AmlCompute.provisioning_configuration(vm_size = 'STANDARD_D2_V2',max_nodes = 4)
        compute_cluster = ComputeTarget.create(ws, cluster, config)
    compute_cluster.wait_for_completion(show_output = True)

#### a) Creating cluster for the first time

In [17]:
cluster_validation()

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


#### b) Calling the function to create cluster again

In [18]:
cluster_validation()

Found existing compute cluster!
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [28]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,randint,choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
        "--max_iter" : choice(5,10,20,40),
        "--C": uniform(0.5,1.0)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval = 2, slack_factor = 0.1,delay_evaluation = 1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = "./",entry_script = 'train.py',compute_target = compute_cluster)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator = est,
                                    hyperparameter_sampling = ps,
                                    policy = policy,
                                    primary_metric_name = 'Accuracy',
                                    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs = 25,
                                    max_concurrent_runs = 4)

In [29]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hdconfig = exp.submit(config = hyperdrive_config)

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


In [30]:
import joblib
# Get your best run and save the model from that run.

RunDetails(hdconfig).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [31]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

url = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
ds = TabularDatasetFactory.from_delimited_files(path = url)

In [32]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###